In [ ]:
import cv2
import sys
import torch
import os
import numpy as np
from matplotlib import pyplot as plt

# Local imports
sys.path.append('..')

from lib.cnn_autoencoder import CnnAutoencoder

In [ ]:
model = "/home/ralampay/Desktop/bottle-good.pth"
state = torch.load(model)
params = state['params']

scale = params.get('scale')
channel_maps = params.get('channel_maps')
padding = params.get('padding')
kernel_size = params.get('kernel_size')
num_channels = params.get('num_channels')
img_width = params.get('img_width')
img_height = params.get('img_height')
device = params.get('device')

model = CnnAutoencoder(
    scale=scale,
    channel_maps=channel_maps,
    padding=padding,
    kernel_size=kernel_size,
    num_channels=num_channels,
    img_width=img_width,
    img_height=img_height
).to(device)

model.load_state_dict(state['state_dict'])
model.optimizer = state['optimizer']

In [ ]:
img_dir = "/home/ralampay/Desktop/good"
images = []

dim = (img_width, img_height)

for img_file in os.listdir(img_dir):
    img_path = os.path.join(img_dir, img_file)
    img = (cv2.resize(cv2.imread(img_path), dim) / 255).astype('float32')
    images.append(img.transpose((2, 0, 1)))
    plt.figure()
    plt.imshow(img)
    


In [ ]:
test_inputs = torch.Tensor(np.array(images)).float().to(device)
predictions = model(test_inputs).detach().cpu().numpy().astype(np.float32)

In [ ]:
for p in predictions:
    img = p.transpose((1, 2, 0))
    plt.figure()
    plt.imshow(img)

In [ ]:
torch.cuda.empty_cache()